In [3]:
import networkx as nx
import numpy as np


np.random.seed(2017)

def construct_adjlist():
    g = nx.barabasi_albert_graph(n=1000, m=3,seed=2017)

    adjlist = nx.generate_adjlist(g, delimiter=",")
    adjlist = [np.array([eval(x)]).flatten() for x in adjlist]

    lens = [len(x) for x in adjlist]
    maxlen = max(lens)
    n = g.number_of_nodes()

    adjlist_m = np.ones([n, maxlen], dtype = np.int32) * (-1)

    for nn in range(n):
        adjlist_m[nn, :lens[nn]] = adjlist[nn]

    return adjlist_m

adjlist_m = construct_adjlist()


def info_spread(adjlist_m, states):  
    N = adjlist_m.shape[0]
    p = 0.2
    lamda = 0.9
    nn_S = np.zeros(N, dtype=np.int32)
    alpha_nS = np.zeros(N, dtype=np.float32)
    new_states = states
    
    for n_i in range(N):
        for n_j in adjlist_m[n_i,:]:
            if n_j == -1:
                continue
            nn_S[n_i] += states[n_j,0]  
            
    alpha_nS = np.tanh(nn_S/2.0, dtype=np.float32) # when n_s = 4, alpha -> 1
    
    for n_i in range(N):
        if states[n_i,2] == 1:
            # n_i is 'R'
            continue
        if states[n_i,1] == 1:
            # n_i is 'I'
            if np.random.random() < alpha_nS[n_i]*p:
                # I-->S
                new_states[n_i,1] = 0
                new_states[n_i,0] = 1
            elif np.random.random() < alpha_nS[n_i]*(1-p):
                # I-->R
                new_states[n_i,1] = 0
                new_states[n_i,2] = 1
        else:
            assert(states[n_i,0] == 1)
            # n_i is 'S'
            if np.random.random() < lamda:
                # S -> R
                new_states[n_i,0] = 0
                new_states[n_i,2] = 1
    
    return new_states


def InfoSpread(adjlist_m, tmax):
    N = adjlist_m.shape[0]
    states = np.zeros([N,3],dtype=np.int32)
    statistics = np.zeros([tmax,3],dtype=np.int32)
     
    # set Ignorants and seeds
    states[:,1] = 1
    states[0:50,0] = 1
    
    for t in range(tmax):
        statistics[t,:] = np.sum(states, axis=0)
        if statistics[t,0] == 0:
            return statistics
        states = info_spread(adjlist_m, states)
    return statistics



    


In [ ]:
%timeit stat = InfoSpread(adjlist_m, 2000)